# Code profiling

In [2]:
# Import libraries
%matplotlib inline
import matplotlib.pyplot as plt
import time
import numpy as np
from numpy import random
import math
import scipy
from scipy import stats
from random import choice
import networkx as nx
import json
from networkx.readwrite import json_graph
import line_profiler
import IPython
ip = IPython.get_ipython()
ip.define_magic('lprun', line_profiler.magic_lprun)

In [3]:
# Load network graph
with open("graph/nc_mini.json", "r") as graph_data:
    graph_data = json.load(graph_data)
    NC_digraph = json_graph.node_link_graph(graph_data)

In [4]:
# Function to print graph
def print_graph(Graph, S1=None):
    plt.figure(figsize=(16,10))
    color_map = {1: 'b', 0: 'r'}
    pos = nx.random_layout(Graph)
    
    if S1:
        nx.draw_networkx(Graph, pos, with_labels=False, node_size=100, node_shape='.',
                linewidth=None, width=0.2, edge_color='y', 
                node_color=[color_map[Graph.node[node]['action']] for node in Graph],
                edgelist=reduce(lambda x,y: x+y,[Graph.edges(node) for node in S1]))
        nx.draw_networkx_nodes(Graph, pos, nodelist=S1, node_color="b", node_size=150, 
                              node_shape="*", label="Initial Set")
        plt.legend()
    else:
        nx.draw_networkx(Graph, pos, with_labels=False, node_size=100, node_shape='.',
                linewidth=None, width=0.2, edge_color='y', 
                 node_color=[color_map[Graph.node[node]['action']] for node in Graph])
        
    plt.xlim(-0.05,1.05)
    plt.ylim(-0.05,1.05)
    plt.xticks([])
    plt.yticks([])
    plt.show()

In [5]:
def activateNodesOpt(detStart, detRandSeed):

    nx.set_node_attributes(NC_digraph, 'activated', False)
    nx.set_node_attributes(NC_digraph, 'explored', False)
    
    if detStart is None:
        if detRandSeed is not None:
            random.seed()
        start = random.choice(NC_digraph.nodes(), 1)
    else:
        start = detStart
    
    nodes = [start[0]]

    activated = 1
    
    NC_digraph.node[start[0]]['activated'] = True
    
    while len(nodes)>0:

        startNode = nodes[0]

        if NC_digraph.node[startNode]['explored']==False:

            NC_digraph.node[startNode]['explored'] = True

            successors = []

            for succNode in NC_digraph.succ[startNode]:

                if NC_digraph.node[succNode]['activated']==False:

                    alpha = NC_digraph[startNode][succNode]['weight']
                    beta = NC_digraph.node[succNode]['review_count']

                    if detRandSeed is not None:
                        random.seed(detRandSeed)
                    randUnif = random.uniform(0,1)
                    if detRandSeed is not None:
                        random.seed(detRandSeed)
                    randBeta = np.sqrt(random.beta(alpha, beta))

                    if randUnif < randBeta:
                        NC_digraph.node[succNode]['activated'] = True
                        successors.append(succNode)
                        activated = activated + 1

        nodes = nodes[1:]
        nodes = nodes + successors

    return activated

def activateNodesLoop(N, rootSeed=None, randNumSeed=None):
    
    if rootSeed is not None and randNumSeed is not None:
        random.seed(rootSeed) 
        detStart = random.choice(NC_digraph.nodes(), 1) # deterministic start node
        detRandSeed = randNumSeed # deterministic random variables
    elif rootSeed is None and randNumSeed is None:
        random.seed()
        detStart = None # stochastic start node
        detRandSeed = None # stochastic random variables
    elif rootSeed is not None and randNumSeed is None:
        random.seed(rootSeed) 
        detStart = random.choice(NC_digraph.nodes(), 1) # deterministic start node
        random.seed()
        detRandSeed = None # stochastic random variables
    elif rootSeed is None and randNumSeed is not None:
        detStart = None # stochastic start node
        detRandSeed = randNumSeed # deterministic random variables

    result = []

    for n in xrange(N):
        result.append(float(activateNodesOpt(detStart, detRandSeed)))
        
    return result

Check runtime of a single iteration - should be ~0.18 ms.

In [6]:
%time result = activateNodesLoop(N=1000, rootSeed=24, randNumSeed=None)

CPU times: user 316 ms, sys: 8.13 ms, total: 324 ms
Wall time: 323 ms


Profile code

In [7]:
%lprun -f activateNodesOpt result = activateNodesLoop(N=1000, rootSeed=24, randNumSeed=None)